In [1]:
import gradio as gr

In [ ]:
def do_something(name):
    return f'Hello, {name}'

In [ ]:
    
app =  gr.Interface(fn = do_something,             # the function to bind to
                     title = 'Say hello to gradio', # title of the interface
                     inputs = 'text',               # type of input(s)
                     outputs = 'text')              # type of output(s)
app.launch()